In [16]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import sys
import time
import random

In [17]:


#directorsDataFrame = pd.read_csv('./unique_directors.xlsx', encoding = "ISO-8859-1")

#directors = directorsDataFrame['uniqueDirectors'].tolist()
#directors = ["John Lasseter", "Joe Johnston", "Forest Whitaker", "Michael Mann"]
#do mod 20 and print out to keep us updated how far it is through

directors = pd.ExcelFile('unique_directors.xlsx')
directors = directors.parse('Sheet1')

directors = directors['directors'].tolist()
len(directors)

3285

In [3]:
#chromeOptions = webdriver.ChromeOptions()
#remove loading imagine during web crawling
#prefs = {'profile.managed_default_content_settings.images':2}
#chromeOptions.add_experimental_option("prefs", prefs)
#chrome_path = "./chromedriver
chrome_path = "./chromedriver"
options = webdriver.ChromeOptions()
#adblock
options.add_argument('--load-extension=./adblock4Sellenium/3.34.0_0')
#remove Javascript to decrease information that will load
options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})
driver = webdriver.Chrome(chrome_path)

In [4]:
#creating a dataframe
directorDataFrame = []

In [5]:
#Finds number of movies from soup
def findNumMovies(soup):
        numCreds = soup.find("div", {"id": "filmo-head-director"})
        if numCreds is None:
             numCreds = soup.find("div", {"id": "filmo-head-director"})
        rawNumOfMovies = numCreds.text
        return rawNumOfMovies

In [6]:
def findWinsAndNoms(rawWinsAndNoms, winsAndNoms):
    if ("wins" in rawWinsAndNoms or "win" in rawWinsAndNoms) and ("nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms):
        numWins = int(winsAndNoms[0])
        numNoms = int(winsAndNoms[1])
    #if they have only one of the two available
    else:
        if "wins" in rawWinsAndNoms or "win" in rawWinsAndNoms:
            numWins = int(winsAndNoms[0])
            numNoms = numWins
        elif "nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms:
            numNoms = int(winsAndNoms[0])
            numWins = 0
    return [numNoms, numWins]

In [7]:
    #under: call function to web scrape the page
    def scrape(directorName, htmlURL):
       # try:
        html_doc = requests.get(htmlURL)
        soup = BeautifulSoup(html_doc.text, 'html.parser')

        awards = soup.findAll("span", {"class": "awards-blurb"})
        # winsAndNoms = re.findall('\s\d+\s', rawWinsAndNoms)
        rawNumOfMovies = findNumMovies(soup)    

        rawNumOfMovies = re.findall('\d+', rawNumOfMovies)
        numMovies = int(rawNumOfMovies[0])
        if len(awards) == 0:
            numWins = 0
            numNoms = 0 
        elif len(awards) == 1 :
            #when no oscars and maybe having just a win / nomination on its own
            rawWinsAndNoms = awards[0].text
            winsAndNoms = re.findall('\d+', rawWinsAndNoms)
            #if director has both wins and nominations
            winsAndNoms = findWinsAndNoms(rawWinsAndNoms, winsAndNoms)
            numNoms = winsAndNoms[0]
            numWins = winsAndNoms[1]
        #When the director has a a special award ontop of other nominations (is an extra visual they add)
        else:                
            rawWinsAndNoms = awards[1].text
            # winsAndNoms = re.findall('\\s\\d+\\s', rawWinsAndNoms)
            winsAndNoms = re.findall('\d+', rawWinsAndNoms)
            #If they have both wins and nominations available
            winsAndNoms = findWinsAndNoms(rawWinsAndNoms, winsAndNoms)
            numNoms = winsAndNoms[0]
            numWins = winsAndNoms[1]
            #
            specialWins = awards[0].text
            specialWins = re.findall('\d+', specialWins)
            specialWins = int(specialWins[0])
            numWins += specialWins
            numNoms += specialWins
        print(directorName, ": Movies: ", numMovies, " Noms: ", numNoms, " Wins: ", numWins)
        directorDataFrame.append({'directorName': directorName, 'directorMovieCount': numMovies,
                                                                 'directorAwardNominations': numNoms, 'directorAwardWins': numWins})
    #except Exception as e:
            #count -= 1
            #print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
            
        #except:
            #print('Error with scraping for' + directorName + '. Skipping to next director. ')



In [8]:
count = len(directors)

driver.get('https://www.imdb.com/?ref_=nv_home')
for name in directors:
        waitTime = random.uniform(0.5,1.5)
        time.sleep(waitTime)
        count -= 1
        if count % 2 == 0:
            print("number of directors remaining: %i" % count)
        #going to main page of IMDB
        #Find search bar, clear and search for director
        try:
            searchBar = driver.find_element_by_id('navbar-query')
            searchBar.clear()
            searchBar.send_keys(name)
            searchBar.send_keys(Keys.RETURN)
            assert "No results found." not in driver.page_source
            try:
                try:
                    #Taking the top director from the list of names that show up from the search
                    personWanted = driver.find_element_by_class_name('findResult')
                    personURL = personWanted.find_element_by_tag_name('a')
                    personURL.click()
                    driver.current_url
                    scrape(name, driver.current_url)
                except:
                    driver.back()
                    personWanted = driver.find_elements_by_class_name('findResult')
                    good = False
                    person = 1
                    #Ensures the person it will be selecting is an director and not another profession
                    while not good:
                        isDirector = personWanted[person].find_element_by_tag_name('small').text
                        if "Director" in isDirector or "Producer" in isDirector:
                            personURL = personWanted[person].find_element_by_tag_name('a')
                            personURL.click()
                            driver.current_url
                            scrape(name, driver.current_url)
                            good = True
                        person = person + 1
                #except instance when a search of an director has a non-driector (ex: show) as the top choice
            except:
                #Reason for doing a whole repeat is because of HTML format where cannot determine if link to click on is director or not
                #Find search bar, clear and search for director
                searchBar = driver.find_element_by_id('navbar-query')
                searchBar.clear()
                searchBar.send_keys(name)
                searchBar.send_keys(Keys.RETURN)
                assert "No results found." not in driver.page_source
                #Taking the top director from the list of names that show up from the search
                moreName = driver.find_element_by_partial_link_text('More name matches')
                moreName.click()
                personWanted = driver.find_element_by_class_name('findResult')
                personURL = personWanted.find_element_by_tag_name('a')
                personURL.click()
                driver.current_url
                scrape(name, driver.current_url)
        #count -= 1
        except Exception as e:
            #count -= 1
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
            print("Director: ", name," not searchable")
            directorDataFrame.append({'directorName': name, 'directorMovieCount': 1,
                                                                 'directorAwardNominations': 0, 'directorAwardWins': 0})



    
directorDataFrame = pd.DataFrame(directorDataFrame, columns = ['directorName', 'directorMovieCount', 'directorAwardNominations', 'directorAwardWins'] )
directorDataFrame.to_csv('directors.csv', index = False)

directorDataFrame

number of directors remaining: 3284
Henry Hathaway : Movies:  67  Noms:  6  Wins:  9
Error on line 55 WebDriverException Message: unknown error: Element <a href="/name/nm0259574/?ref_=fn_nm_nm_1">...</a> is not clickable at point (138, 399). Other element would receive the click: <iframe id="floater_8393073" name="floater_8393073" width="140px" height="64px" frameborder="0" scrolling="no" style="margin-top: 160px; z-index: 1500; position: absolute;"></iframe>
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.11.6 x86_64)

Director:  Benedikt Erlingsson  not searchable
number of directors remaining: 3282
Becky Smith : Movies:  16  Noms:  2  Wins:  6
Drew Heriot : Movies:  5  Noms:  0  Wins:  0
number of directors remaining: 3280
Jorge Blanco : Movies:  1  Noms:  1  Wins:  1
Dennis Hopper : Movies:  9  Noms:  17  Wins:  25
number of directors remaining: 3278
Luc Jacquet : Movies:  8  Noms:  10  

Ava DuVernay : Movies:  21  Noms:  46  Wins:  53
number of directors remaining: 3168
Stephen Milburn Anderson : Movies:  4  Noms:  2  Wins:  0
Aku Louhimies : Movies:  37  Noms:  17  Wins:  23
number of directors remaining: 3166
Randall Rubin : Movies:  1  Noms:  1  Wins:  1
Andrew Erwin : Movies:  11  Noms:  1  Wins:  3
number of directors remaining: 3164
Wim Wenders : Movies:  65  Noms:  66  Wins:  89
Abhishek Chaubey : Movies:  4  Noms:  6  Wins:  1
number of directors remaining: 3162
Nicolas Winding Refn : Movies:  15  Noms:  51  Wins:  28
Jonathan Glazer : Movies:  20  Noms:  35  Wins:  8
number of directors remaining: 3160
Douglas Cheek : Movies:  3  Noms:  1  Wins:  1
Tom Mankiewicz : Movies:  6  Noms:  1  Wins:  0
number of directors remaining: 3158
Niels Arden Oplev : Movies:  23  Noms:  18  Wins:  17
Ryan Gosling : Movies:  1  Noms:  142  Wins:  35
number of directors remaining: 3156
Shane Acker : Movies:  6  Noms:  6  Wins:  6
Liv Ullmann : Movies:  7  Noms:  22  Wins:  52
n

number of directors remaining: 3048
Joseph Kosinski : Movies:  6  Noms:  1  Wins:  0
Arman Gevorgyan : Movies:  4  Noms:  0  Wins:  0
number of directors remaining: 3046
Sergey Ivanov : Movies:  1  Noms:  1  Wins:  0
Error on line 57 AttributeError 'NoneType' object has no attribute 'text'
Director:  Bill Wilson  not searchable
number of directors remaining: 3044
Mike Mitchell : Movies:  15  Noms:  5  Wins:  5
Justin Barber : Movies:  2  Noms:  3  Wins:  0
number of directors remaining: 3042
Samuel Armstrong : Movies:  6  Noms:  1  Wins:  1
William Sachs : Movies:  12  Noms:  1  Wins:  3
number of directors remaining: 3040
Michael Lehmann : Movies:  48  Noms:  7  Wins:  4
Mike Gunther : Movies:  11  Noms:  5  Wins:  5
number of directors remaining: 3038
Galt Niederhoffer : Movies:  4  Noms:  0  Wins:  0
Franc Roddam : Movies:  13  Noms:  3  Wins:  1
number of directors remaining: 3036
Michael Cimino : Movies:  8  Noms:  14  Wins:  14
Dick Huemer : Movies:  35  Noms:  2  Wins:  2
number

Edoardo Leo : Movies:  6  Noms:  11  Wins:  16
number of directors remaining: 2926
Frank Capra : Movies:  58  Noms:  15  Wins:  12
Steve Boyum : Movies:  31  Noms:  5  Wins:  0
number of directors remaining: 2924
Jim Field Smith : Movies:  13  Noms:  1  Wins:  0
Richard Benjamin : Movies:  22  Noms:  7  Wins:  2
number of directors remaining: 2922
Busby Berkeley : Movies:  31  Noms:  5  Wins:  5
Frank Henenlotter : Movies:  13  Noms:  3  Wins:  3
number of directors remaining: 2920
Fyodor Bondarchuk : Movies:  11  Noms:  22  Wins:  9
Richard Boleslawski : Movies:  24  Noms:  1  Wins:  1
number of directors remaining: 2918
Til Schweiger : Movies:  16  Noms:  6  Wins:  26
Brad Bird : Movies:  11  Noms:  31  Wins:  32
number of directors remaining: 2916
Ram Madhvani : Movies:  4  Noms:  3  Wins:  5
Jared Hess : Movies:  13  Noms:  4  Wins:  1
number of directors remaining: 2914
Error on line 57 UnboundLocalError local variable 'numNoms' referenced before assignment
Director:  Will Gluck  

Raja Menon : Movies:  4  Noms:  1  Wins:  0
Fatih Akin : Movies:  19  Noms:  40  Wins:  47
number of directors remaining: 2806
Michael Winner : Movies:  42  Noms:  1  Wins:  0
Alain Chabat : Movies:  8  Noms:  5  Wins:  2
number of directors remaining: 2804
Zhang Yimou : Movies:  32  Noms:  51  Wins:  66
Roger Donaldson : Movies:  27  Noms:  7  Wins:  4
number of directors remaining: 2802
Shim Hyung-Rae : Movies:  10  Noms:  1  Wins:  0
Error on line 57 UnboundLocalError local variable 'numNoms' referenced before assignment
Director:  Paiman Kalayeh  not searchable
number of directors remaining: 2800
Brandon Trost : Movies:  8  Noms:  2  Wins:  2
Rowan Joffe : Movies:  5  Noms:  7  Wins:  4
number of directors remaining: 2798
George Nolfi : Movies:  7  Noms:  3  Wins:  0
Jonathan Hensleigh : Movies:  3  Noms:  2  Wins:  0
number of directors remaining: 2796
Walt Becker : Movies:  10  Noms:  1  Wins:  0
Oren Moverman : Movies:  4  Noms:  16  Wins:  12
number of directors remaining: 2794

...

number of directors remaining: 384
Stig Bergqvist : Movies:  13  Noms:  2  Wins:  10
Allan A. Goldstein : Movies:  29  Noms:  2  Wins:  2
number of directors remaining: 382
Error on line 57 UnboundLocalError local variable 'numNoms' referenced before assignment
Director:  Sam Wanamaker  not searchable
George Armitage : Movies:  7  Noms:  2  Wins:  2
number of directors remaining: 380
David McNally : Movies:  5  Noms:  0  Wins:  0
Bob Fosse : Movies:  8  Noms:  11  Wins:  14
number of directors remaining: 378
Matt Dillon : Movies:  3  Noms:  25  Wins:  22
Katherine Dieckmann : Movies:  10  Noms:  2  Wins:  0
number of directors remaining: 376
Barry Sonnenfeld : Movies:  23  Noms:  11  Wins:  7
Elliot Greenebaum : Movies:  1  Noms:  4  Wins:  4
number of directors remaining: 374
David Yates : Movies:  27  Noms:  20  Wins:  16
Duncan Tucker : Movies:  2  Noms:  5  Wins:  7
number of directors remaining: 372
Anthony Byrne : Movies:  17  Noms:  5  Wins:  0
Ariel Vromen : Movies:  7  Noms:  

number of directors remaining: 262
Brett Ratner : Movies:  54  Noms:  6  Wins:  8
F.W. Murnau : Movies:  21  Noms:  3  Wins:  3
number of directors remaining: 260
Rupert Wyatt : Movies:  11  Noms:  7  Wins:  2
Jon M. Chu : Movies:  22  Noms:  1  Wins:  0
number of directors remaining: 258
John Hughes : Movies:  8  Noms:  2  Wins:  1
Patricia Cardoso : Movies:  14  Noms:  1  Wins:  8
number of directors remaining: 256
Rolf de Heer : Movies:  20  Noms:  49  Wins:  41
Seth Gordon : Movies:  22  Noms:  1  Wins:  3
number of directors remaining: 254
Zak Levitt : Movies:  5  Noms:  1  Wins:  1
Stephen Gaghan : Movies:  6  Noms:  18  Wins:  12
number of directors remaining: 252
Matt Reeves : Movies:  13  Noms:  8  Wins:  2
Daniel Stamm : Movies:  9  Noms:  2  Wins:  2
number of directors remaining: 250
Tim Burton : Movies:  38  Noms:  72  Wins:  23
Fede Alvarez : Movies:  10  Noms:  6  Wins:  1
number of directors remaining: 248
Eric Schaeffer : Movies:  13  Noms:  8  Wins:  8
Christian E. Ch

number of directors remaining: 140
Nalan Kumarasamy : Movies:  3  Noms:  1  Wins:  1
Andrea Arnold : Movies:  12  Noms:  29  Wins:  61
number of directors remaining: 138
Nick Park : Movies:  8  Noms:  30  Wins:  58
Karen Moncrieff : Movies:  11  Noms:  6  Wins:  4
number of directors remaining: 136
Evgeniy Bedarev : Movies:  5  Noms:  0  Wins:  0
Louis C.K. : Movies:  24  Noms:  85  Wins:  26
number of directors remaining: 134
Franc. Reyes : Movies:  4  Noms:  1  Wins:  1
Jack Smight : Movies:  66  Noms:  3  Wins:  1
number of directors remaining: 132
Kelly Makin : Movies:  50  Noms:  16  Wins:  5
Robert Iscove : Movies:  63  Noms:  6  Wins:  3
number of directors remaining: 130
Rudolf van den Berg : Movies:  17  Noms:  4  Wins:  8
Nick Hamm : Movies:  19  Noms:  3  Wins:  3
number of directors remaining: 128
Michael Ritchie : Movies:  38  Noms:  4  Wins:  2
Arie Posin : Movies:  3  Noms:  1  Wins:  2
number of directors remaining: 126
Norman Jewison : Movies:  44  Noms:  25  Wins:  35

number of directors remaining: 18
Irvin Kershner : Movies:  32  Noms:  3  Wins:  6
"John OHagan" : Movies:  2  Noms:  2  Wins:  2
number of directors remaining: 16
Antonio Banderas : Movies:  3  Noms:  41  Wins:  41
Richard Wallace : Movies:  63  Noms:  1  Wins:  1
number of directors remaining: 14
Alex Cox : Movies:  22  Noms:  8  Wins:  4
Haile Gerima : Movies:  12  Noms:  5  Wins:  11
number of directors remaining: 12
Michael G. Bartlett : Movies:  4  Noms:  0  Wins:  0
Martha Coolidge : Movies:  52  Noms:  10  Wins:  8
number of directors remaining: 10
Jonathan Nossiter : Movies:  9  Noms:  5  Wins:  4
Error on line 57 AttributeError 'NoneType' object has no attribute 'text'
Director:  John A. Davis  not searchable
number of directors remaining: 8
Paul Flaherty : Movies:  11  Noms:  19  Wins:  5
Wes Ball : Movies:  6  Noms:  2  Wins:  2
number of directors remaining: 6
Bernardo Bertolucci : Movies:  25  Noms:  34  Wins:  49
Mora Stephens : Movies:  5  Noms:  1  Wins:  2
number of d

,directorName,directorMovieCount,directorAwardNominations,directorAwardWins
0,Henry Hathaway,67,6,9
1,Benedikt Erlingsson,1,0,0
2,Becky Smith,16,2,6
3,Drew Heriot,5,0,0
4,Jorge Blanco,1,1,1
5,Dennis Hopper,9,17,25
6,Luc Jacquet,8,10,5
7,Steve Martino,9,7,1
8,Christopher Nolan,14,218,138
9,Davis Guggenheim,38,15,19


In [12]:
writer = pd.ExcelWriter('directors.xlsx', engine='xlsxwriter')

directorDataFrame.to_excel(writer, 'Sheet1')
writer.save()

In [13]:
len(directorDataFrame
   )

3285

In [14]:
len(directors)

3285